In [1]:
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
import numpy as np
from matplotlib import pyplot
from math import sqrt
from pytz import timezone

In [2]:
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import MinMaxScaler, Normalizer, RobustScaler
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Activation
from keras import optimizers

Using TensorFlow backend.


In [3]:
import talib

In [4]:
# Fix the random seed to reproducibility
np.random.seed(7)

In [5]:
def get_X_data():
    import dovahkiin as dk
    dp = dk.DataParser()
    X = dp.get_data("cu")
    return X

In [6]:
X = get_X_data()
X = X["close"]
y = pd.Series(talib.SMA(X.values, 40), index=X.index)

In [7]:
X = X["2012":]
y = y["2012":]

In [8]:
# Use difference in X as a predicator
# X = X.diff()
X = X.fillna(0)
y = y.fillna(0)

In [22]:
def timeseries_to_supervised(raw_time_series, lag):
    p = {}
    for i in range(1, lag+1):
        p["{}".format(i)] = raw_time_series.shift(i).fillna(0)
    p["0"] = raw_time_series
    
    if type(raw_time_series) is pd.Series:
        supervised_data = pd.DataFrame(p)
    else:
        supervised_data = pd.Panel(p)
    return supervised_data

In [23]:
def non_shuffling_train_test_split(X, y, test_size=0.2):
    i = int((1 - test_size) * X.shape[0]) + 1
    X_train, X_test = np.split(X, [i])
    y_train, y_test = np.split(y, [i])
    return X_train, X_test, y_train, y_test

In [24]:
def create_supervised_X(raw_time_series, lag):
    supervised_X = timeseries_to_supervised(X, 10)
    swaped_supervised_X = supervised_X.swapaxes(0, 1)
    return swaped_supervised_X

In [29]:
supervised_X.shape

(11, 500310)

In [25]:
supervised_X = create_supervised_X(X, lag=20)
supervised_X = supervised_X.fillna(0)
supervised_X_values = supervised_X.values

In [14]:
supervised_X.items[350218:][0]

Timestamp('2016-03-01 21:28:00+0800', tz='Asia/Shanghai')

In [15]:
X_train, X_test, y_train, y_test = non_shuffling_train_test_split(X, y, test_size=0.3)

In [16]:
X_train = X_train[218:]
y_train = y_train[218:]
print(len(X_train))

350000


In [17]:
X_test = X_test[92:]
y_test = y_test[92:]
print(len(X_test))

150000


In [18]:
def scale(X_train, X_test):
    # scaler = MinMaxScaler(feature_range=(-1, 1), copy=True)
    scaler = Normalizer()
    # scaler = RobustScaler()
    scaler = scaler.fit(X_train, y_train)
    train_scaled = pd.DataFrame(scaler.transform(X_train), index=X_train.index, columns=X_train.columns)
    test_scaled = pd.DataFrame(scaler.transform(X_test), index=X_test.index, columns=X_test.columns)
    return scaler, train_scaled, test_scaled

In [19]:
def inverse_scale(scaler, X_train_scaled, X_test_scaled):
    X_train = scaler.inverse_transform(X_train_scaled)
    X_test = scaler.inverse_transform(X_test_scaled)
    X_train = pd.DataFrame(scaler.transform(X_train), index=X_train_scaled.index, columns=X_train_scaled.columns)
    X_test = pd.DataFrame(scaler.transform(X_test), index=X_test_scaled.index, columns=X_test_scaled.columns)
    return scaler, X_train, X_test

In [20]:
scaler, X_train_scaled, X_test_scaled = scale(X_train, X_test)

In [21]:
time_series_step = 100
features = X_train.shape[1]

In [22]:
X_train_supervised_scaled = timeseries_to_supervised(X_train_scaled, time_series_step)
X_train_supervised_scaled = X_train_supervised_scaled.swapaxes(0, 1)

X_test_supervised_scaled = timeseries_to_supervised(X_test_scaled, time_series_step)
X_test_supervised_scaled = X_test_supervised_scaled.swapaxes(0, 1)

In [23]:
batch_size = 1000

In [24]:
yscaler = MinMaxScaler()
yscaler.fit(y_train)
y_train_scaled = yscaler.transform(y_train)

/usr/local/lib/python3.6/site-packages/sklearn/preprocessing/data.py:321: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  warnings.warn(DEPRECATION_MSG_1D, DeprecationWarning)
/usr/local/lib/python3.6/site-packages/sklearn/preprocessing/data.py:356: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  warnings.warn(DEPRECATION_MSG_1D, DeprecationWarning)


In [25]:
def LSTM_Model():
    model = Sequential()
    model.add(
        LSTM(128, batch_input_shape=(batch_size, time_series_step+1, features), stateful=True, 
         return_sequences=True, 
         activation="relu"
        ))
    model.add(LSTM(32, activation="relu", stateful=True))
    model.add(Dense(128))
    model.add(Dense(1))
    
    return model


In [29]:
def FullyConnected_Model():
    model = Sequential()
    model.add(Dense(128), input_shape)
    model.add(Dense(64))
    model.add(Dense(32))
    model.add(Dense(1))
    return model

In [31]:
sgd = optimizers.SGD(lr=1e-10, decay=1e-6, momentum=0.9, nesterov=True)

In [32]:
model = LSTM_Model()
model.compile(loss="mean_squared_error", optimizer=sgd, metrics=[ 'mse'])

In [30]:
model = FullyConnected_Model()
model.compile(loss="mean_squared_error", optimizer=sgd, metrics=[ 'mse'])

ValueError: The first layer in a Sequential model must get an `input_shape` or `batch_input_shape` argument.

In [ ]:
num_epochs = 1
for i in range(num_epochs):
    model.fit(
        X_train_supervised_scaled.values, 
        y_train,
        epochs=1,
        batch_size=batch_size,
        verbose=1,
        shuffle=False,
        validation_split=0.2
    )
    model.reset_states()

Train on 280000 samples, validate on 70000 samples
Epoch 1/1
 56000/280000 [=====>........................] - ETA: 713s - loss: 3287778445.7143 - mean_squared_error: 3287778445.7143 - acc: 0.0000e+00 